# InstanceTypes-wise operation

This notebook demostrates common instancetypes-wise operations by using SDK.

## Prerequisites

You need

- an SDK environment set up with the administrator priviledge. Check "*environment setup.ipynb*" for the detail.

In [ ]:
PRIMEHUB_CLUSTER = 'your_PrimeHub_domain'

**Check if the configuration exists. If not, you need to login to retrieve the token and input it in the prompt. The configuration will be generated at**`~/.primehub/config.json`.

In [ ]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print(f"PrimeHub Python SDK {ph.version.version()} environment is ready, you are good to go.")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("Failed to retrieve the information from PrimeHub cluster, please check the configuration.")

## ToC

- [Fundamental Operations (CRUD)](#fundamental)
- [Common Operations](#common)

---
## Fundamental Operations <a class="anchor" id="fundamental"></a>

In [ ]:
# Get the group id by the given group name
group_name = 'Continental'
groups = ph.admin.groups.list()
for group in groups:
    if group['name'] == group_name:
        specified_group = group
        
group_detail = ph.admin.groups.get(specified_group['id'])
group_detail

### Create InstanceType

Method: `ph.admin.instancetypes.create(instancetype_configuration)`

You can opt for adding a *global instancetype* or an *instancetype connecting specified groups*.

In [ ]:
instancetype_config_global = {
  "name": "basic-global",
  "displayName": "Basic-Global by SDK",
  "description": "1 CPU / 1GB Mem",
  "cpuLimit": 1,
  "memoryLimit": 1,
  "global": True
}

instancetype_config_with_specified_groups = {
  "name": "basic-group",
  "displayName": "Basic-Groups by SDK",
  "description": "1 CPU / 1GB Mem",
  "cpuLimit": 1,
  "memoryLimit": 1,
  "groups": {
    "connect": [
      {
        "id": specified_group['id'],
      }
    ]
  }
}

In [ ]:
instancetype = ph.admin.instancetypes.create(instancetype_config_with_specified_groups)
instancetype

### Get InstanceType Detail

Method: `ph.admin.instancetypes.get(instancetype_id)`

In [ ]:
ph.admin.instancetypes.get(instancetype['id'])

### Update InstanceType Configuration

Method: `ph.admin.instancetypes.update(instancetype_id, config)`

In [ ]:
updating_config = {
  "description": "4 CPU / 4GB Mem",
  "gpuLimit": 2,
}

In [ ]:
ph.admin.instancetypes.update(instancetype['id'], updating_config)

#Verify
ph.admin.instancetypes.get(instancetype['id'])

### Get connected Groups of InstanceTypes

Method: `ph.admin.instancetypes.list_groups('instancetype_id')`

In [ ]:
ph.admin.instancetypes.list_groups(instancetype['id'])

### List All of InstaceTypes

Method: `ph.admin.instancetypes.list()`

In [ ]:
all_images = ph.admin.instancetypes.list()
for image in all_images:
    print(image)

### Delete InstanceType

Method: `ph.admin.instancetype.delete(instancetype_id)`

In [ ]:
ph.admin.instancetypes.delete(instancetype['id'])

---

## Common Operations <a class="anchor" id="common"></a>

### Connect/Disconnect InstanceType to Group

Method: 
- `ph.admin.instancetypes.add_group('instancetype_id', 'group_id')`
- `ph.admin.instancetypes.remove_group('instancetype_id', 'group_id')`

In [ ]:
instancetype_id = 'basic-group'

ph.admin.instancetypes.add_group(instancetype_id, specified_group['id'])

#Verify
ph.admin.instancetypes.list_groups(instancetype_id)

In [ ]:
ph.admin.instancetypes.remove_group(instancetype_id, specified_group['id'])

#Verify
ph.admin.instancetypes.list_groups(instancetype_id)

### Add/Remove Toleration on InstanceType

By `ph.admin.instancetypes.update()` you can add a tolerations on the instancetype.

In [ ]:
# Add
toleration_config = {
  "tolerations": {
    "set": [
      {
        "operator": "Equal",
        "effect": "NoSchedule",
        "key": "nvidia.com/gpu",
        "value": "v100"
      }
    ]
  }
}
ph.admin.instancetypes.update(instancetype['id'], toleration_config)

ph.admin.instancetypes.get(instancetype['id'])

In [ ]:
# Remove, update the config with an empty spec []
toleration_config = {
  "tolerations": {
    "set": []
  }
}
ph.admin.instancetypes.update(instancetype['id'], toleration_config)

ph.admin.instancetypes.get(instancetype['id'])

### Add/Remove Node Selector on InstanceType

By `ph.admin.instancetypes.update()` you can add a NodeSelector on instancetype.

In [ ]:
# Add
nodeselector_config = {
  "nodeSelector": {"zone": "staging"}
}

ph.admin.instancetypes.update(instancetype['id'], nodeselector_config)
ph.admin.instancetypes.get(instancetype['id'])

In [ ]:
# Remove, update the config with None
nodeselector_config = {
  "nodeSelector": None
}

ph.admin.instancetypes.update(instancetype['id'], nodeselector_config)
ph.admin.instancetypes.get(instancetype['id'])